In [28]:
import csv
import boto3
import pandas as pd
import cv2
import matplotlib.pyplot as plt 
import json
import sys
import time

In [2]:
with open("credentials.csv","r") as input:
    next(input)
    reader=csv.reader(input)
    for line in reader:
        access_key_id=line[2]
        secret_access_key=line[3]

In [3]:
photo="hotairbaloon.jpg"

In [4]:
client = boto3.client('rekognition', aws_access_key_id = access_key_id, aws_secret_access_key = secret_access_key, region_name='us-east-1')

In [22]:
with open(photo,'rb') as source_image:
    source_bytes=source_image.read()

In [23]:
response=client.detect_labels(Image={'Bytes':source_bytes},MaxLabels=2,MinConfidence=95)
print(response)

{'Labels': [{'Name': 'Ball', 'Confidence': 99.47676086425781, 'Instances': [], 'Parents': []}, {'Name': 'Balloon', 'Confidence': 99.47676086425781, 'Instances': [{'BoundingBox': {'Width': 0.4699590802192688, 'Height': 0.7395995855331421, 'Left': 0.07522430270910263, 'Top': 0.09918608516454697}, 'Confidence': 99.47676086425781}], 'Parents': [{'Name': 'Ball'}]}], 'LabelModelVersion': '2.0', 'ResponseMetadata': {'RequestId': '8479da7d-880f-40e5-b93c-7b66a37f6d55', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Sun, 26 Jan 2020 02:48:17 GMT', 'x-amzn-requestid': '8479da7d-880f-40e5-b93c-7b66a37f6d55', 'content-length': '362', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [10]:
pic='sum.jpeg'

In [11]:
response=client.detect_labels(Image={'S3Object':{'Bucket':'2020sd','Name':pic}},MaxLabels=10)
print(response)

{'Labels': [{'Name': 'Person', 'Confidence': 99.26679229736328, 'Instances': [{'BoundingBox': {'Width': 0.7147886157035828, 'Height': 0.6959484815597534, 'Left': 0.18300040066242218, 'Top': 0.2179289609193802}, 'Confidence': 99.26679229736328}], 'Parents': []}, {'Name': 'Pc', 'Confidence': 93.47022247314453, 'Instances': [], 'Parents': [{'Name': 'Electronics'}, {'Name': 'Computer'}]}, {'Name': 'Electronics', 'Confidence': 93.47022247314453, 'Instances': [], 'Parents': []}, {'Name': 'Computer', 'Confidence': 93.47022247314453, 'Instances': [], 'Parents': [{'Name': 'Electronics'}]}, {'Name': 'Laptop', 'Confidence': 87.47058868408203, 'Instances': [{'BoundingBox': {'Width': 0.4931282103061676, 'Height': 0.15939590334892273, 'Left': 0.011257215403020382, 'Top': 0.5771797299385071}, 'Confidence': 83.18744659423828}], 'Parents': [{'Name': 'Pc'}, {'Name': 'Electronics'}, {'Name': 'Computer'}]}, {'Name': 'Female', 'Confidence': 85.0995864868164, 'Instances': [], 'Parents': [{'Name': 'Person'}]

In [24]:
def create_collection(collection_id):

#     client=boto3.client('rekognition')

    #Create a collection
    print('Creating collection:' + collection_id)
    response=client.create_collection(CollectionId=collection_id)
    print('Collection ARN: ' + response['CollectionArn'])
    print('Status code: ' + str(response['StatusCode']))
    print('Done...')

In [25]:
collection_id='Collection'
create_collection(collection_id)

Creating collection:Collection
Collection ARN: aws:rekognition:us-east-1:749661538333:collection/Collection
Status code: 200
Done...


In [26]:
def add_faces_to_collection(bucket,photo,collection_id):

    response=client.index_faces(CollectionId=collection_id,
                                Image={'S3Object':{'Bucket':bucket,'Name':photo}},
                                ExternalImageId=photo,
                                MaxFaces=1,
                                QualityFilter="AUTO",
                                DetectionAttributes=['ALL'])

    print ('Results for ' + photo)
    print('Faces indexed:')
    for faceRecord in response['FaceRecords']:
        print('  Face ID: ' + faceRecord['Face']['FaceId'])
        print('  Location: {}'.format(faceRecord['Face']['BoundingBox']))

    print('Faces not indexed:')
    for unindexedFace in response['UnindexedFaces']:
        print(' Location: {}'.format(unindexedFace['FaceDetail']['BoundingBox']))
        print(' Reasons:')
        for reason in unindexedFace['Reasons']:
            print('   ' + reason)
    return len(response['FaceRecords'])

In [27]:
bucket='2020sd'
collection_id='Collection'
photo='sum.jpeg'
indexed_faces_count=add_faces_to_collection(bucket, photo, collection_id)
print("Faces indexed count: " + str(indexed_faces_count))

Results for sum.jpeg
Faces indexed:
  Face ID: f4a716e2-3825-413e-9158-99f9a77efed7
  Location: {'Width': 0.26034557819366455, 'Height': 0.17324276268482208, 'Left': 0.36181923747062683, 'Top': 0.26396483182907104}
Faces not indexed:
Faces indexed count: 1


In [52]:
Obj=[]

In [55]:
class VideoDetect:
    jobId = ''
    rek = boto3.client('rekognition',aws_access_key_id = access_key_id, aws_secret_access_key = secret_access_key,region_name='us-east-1')
    sqs = boto3.client('sqs',aws_access_key_id = access_key_id, aws_secret_access_key = secret_access_key,region_name='us-east-1')
    sns = boto3.client('sns',aws_access_key_id = access_key_id, aws_secret_access_key = secret_access_key,region_name='us-east-1')
    
    roleArn = ''
    bucket = ''
    video = ''
    startJobId = ''

    sqsQueueUrl = ''
    snsTopicArn = ''
    processType = ''

    def __init__(self, role, bucket, video):    
        self.roleArn = role
        self.bucket = bucket
        self.video = video

    def GetSQSMessageSuccess(self):

        jobFound = False
        succeeded = False
    
        dotLine=0
        while jobFound == False:
            sqsResponse = self.sqs.receive_message(QueueUrl=self.sqsQueueUrl, MessageAttributeNames=['ALL'],
                                          MaxNumberOfMessages=10)

            if sqsResponse:
                
                if 'Messages' not in sqsResponse:
                    if dotLine<40:
                        print('.', end='')
                        dotLine=dotLine+1
                    else:
                        print()
                        dotLine=0    
                    sys.stdout.flush()
                    time.sleep(5)
                    continue

                for message in sqsResponse['Messages']:
                    notification = json.loads(message['Body'])
                    rekMessage = json.loads(notification['Message'])
                    print(rekMessage['JobId'])
                    print(rekMessage['Status'])
                    if rekMessage['JobId'] == self.startJobId:
                        print('Matching Job Found:' + rekMessage['JobId'])
                        jobFound = True
                        if (rekMessage['Status']=='SUCCEEDED'):
                            succeeded=True

                        self.sqs.delete_message(QueueUrl=self.sqsQueueUrl,
                                       ReceiptHandle=message['ReceiptHandle'])
                    else:
                        print("Job didn't match:" +
                              str(rekMessage['JobId']) + ' : ' + self.startJobId)
                    # Delete the unknown message. Consider sending to dead letter queue
                    self.sqs.delete_message(QueueUrl=self.sqsQueueUrl,
                                   ReceiptHandle=message['ReceiptHandle'])


        return succeeded

    def StartLabelDetection(self):
        response=self.rek.start_label_detection(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
            NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn': self.snsTopicArn})

        self.startJobId=response['JobId']
        print('Start Job Id: ' + self.startJobId)


    def GetLabelDetectionResults(self):
        maxResults = 7
        paginationToken = ''
        finished = False
        obj=[]

        while finished == False:
            response = self.rek.get_label_detection(JobId=self.startJobId,
                                            MaxResults=maxResults,
                                            NextToken=paginationToken,
                                            SortBy='TIMESTAMP')

#             print('Codec: ' + response['VideoMetadata']['Codec'])
#             print('Duration: ' + str(response['VideoMetadata']['DurationMillis']))
#             print('Format: ' + response['VideoMetadata']['Format'])
#             print('Frame rate: ' + str(response['VideoMetadata']['FrameRate']))
#             print()

            for labelDetection in response['Labels']:
                label=labelDetection['Label']

#                 print("Timestamp: " + str(labelDetection['Timestamp']))
                if(label['Confidence']>=97.0):
                    print("   Label: " + label['Name'])
                    obj.append(label['Name'])
#                 print("   Confidence: " +  str(label['Confidence']))
#                 print("   Instances:")
#                 for instance in label['Instances']:
#                     print ("      Confidence: " + str(instance['Confidence']))
#                     print ("      Bounding box")
#                     print ("        Top: " + str(instance['BoundingBox']['Top']))
#                     print ("        Left: " + str(instance['BoundingBox']['Left']))
#                     print ("        Width: " +  str(instance['BoundingBox']['Width']))
#                     print ("        Height: " +  str(instance['BoundingBox']['Height']))
#                     print()
#                 print()
#                 print ("   Parents:")
#                 for parent in label['Parents']:
#                     print ("      " + parent['Name'])
#                 print ()

                if 'NextToken' in response:
                    paginationToken = response['NextToken']
                else:
                    finished = True
        return obj
       
    
    def CreateTopicandQueue(self):
      
        millis = str(int(round(time.time() * 1000)))

        #Create SNS topic
        
        snsTopicName="AmazonRekognitionExample" + millis

        topicResponse=self.sns.create_topic(Name=snsTopicName)
        self.snsTopicArn = topicResponse['TopicArn']

        #create SQS queue
        sqsQueueName="AmazonRekognitionQueue" + millis
        self.sqs.create_queue(QueueName=sqsQueueName)
        self.sqsQueueUrl = self.sqs.get_queue_url(QueueName=sqsQueueName)['QueueUrl']
 
        attribs = self.sqs.get_queue_attributes(QueueUrl=self.sqsQueueUrl,
                                                    AttributeNames=['QueueArn'])['Attributes']
                                        
        sqsQueueArn = attribs['QueueArn']

        # Subscribe SQS queue to SNS topic
        self.sns.subscribe(
            TopicArn=self.snsTopicArn,
            Protocol='sqs',
            Endpoint=sqsQueueArn)

        #Authorize SNS to write SQS queue 
        policy = """{{
  "Version":"2012-10-17",
  "Statement":[
    {{
      "Sid":"MyPolicy",
      "Effect":"Allow",
      "Principal" : {{"AWS" : "*"}},
      "Action":"SQS:SendMessage",
      "Resource": "{}",
      "Condition":{{
        "ArnEquals":{{
          "aws:SourceArn": "{}"
        }}
      }}
    }}
  ]
}}""".format(sqsQueueArn, self.snsTopicArn)
 
        response = self.sqs.set_queue_attributes(
            QueueUrl = self.sqsQueueUrl,
            Attributes = {
                'Policy' : policy
            })

    def DeleteTopicandQueue(self):
        self.sqs.delete_queue(QueueUrl=self.sqsQueueUrl)
        self.sns.delete_topic(TopicArn=self.snsTopicArn)


In [56]:
roleArn = 'arn:aws:iam::749661538333:role/CURole'   
bucket = '2020sd'
video = 'cuhack.mp4'
obj=[]
analyzer=VideoDetect(roleArn, bucket,video)
analyzer.CreateTopicandQueue()

analyzer.StartLabelDetection()
if analyzer.GetSQSMessageSuccess()==True:
    obj=analyzer.GetLabelDetectionResults()
    
analyzer.DeleteTopicandQueue()

Start Job Id: 5cad0f34a4ee86a9cd186b4276d65617fbc22ca2d80bbf8eac6d44d435c2d64c
........................................
..5cad0f34a4ee86a9cd186b4276d65617fbc22ca2d80bbf8eac6d44d435c2d64c
SUCCEEDED
Matching Job Found:5cad0f34a4ee86a9cd186b4276d65617fbc22ca2d80bbf8eac6d44d435c2d64c
   Label: Computer
   Label: Computer Hardware
   Label: Computer Keyboard
   Label: Electronics
   Label: Furniture
   Label: Hardware
   Label: Human
   Label: Keyboard
   Label: Laptop
   Label: Pc
   Label: Person
   Label: Computer
   Label: Computer Hardware
   Label: Computer Keyboard
   Label: Electronics
   Label: Furniture
   Label: Hardware
   Label: Human
   Label: Keyboard
   Label: Laptop
   Label: Pc
   Label: Person
   Label: Human
   Label: Person
   Label: Furniture
   Label: Human
   Label: Person
   Label: Chair
   Label: Furniture
   Label: Chair
   Label: Furniture
   Label: Display
   Label: Electronics
   Label: Human
   Label: Monitor
   Label: Person
   Label: Screen
   Label: Display

In [57]:
print(obj)

['Computer', 'Computer Hardware', 'Computer Keyboard', 'Electronics', 'Furniture', 'Hardware', 'Human', 'Keyboard', 'Laptop', 'Pc', 'Person', 'Computer', 'Computer Hardware', 'Computer Keyboard', 'Electronics', 'Furniture', 'Hardware', 'Human', 'Keyboard', 'Laptop', 'Pc', 'Person', 'Human', 'Person', 'Furniture', 'Human', 'Person', 'Chair', 'Furniture', 'Chair', 'Furniture', 'Display', 'Electronics', 'Human', 'Monitor', 'Person', 'Screen', 'Display', 'Electronics', 'Human', 'Monitor', 'Person', 'Screen', 'Display', 'Electronics', 'Human', 'Monitor', 'Person', 'Screen', 'Human', 'Person', 'Human', 'Indoors', 'Lobby', 'Person', 'Room', 'Couch', 'Furniture', 'Human', 'Indoors', 'Lobby', 'Person', 'Room']


In [60]:
count = obj.count('Person') #count of humans
newlist=obj
stopwords = ['Person', 'Human']

In [62]:
newlist=[word for word in obj if word not in stopwords ]

In [65]:
res = [i for n, i in enumerate(newlist) if i not in newlist[:n]] 
print(res) # res has list of words

['Computer', 'Computer Hardware', 'Computer Keyboard', 'Electronics', 'Furniture', 'Hardware', 'Keyboard', 'Laptop', 'Pc', 'Chair', 'Display', 'Monitor', 'Screen', 'Indoors', 'Lobby', 'Room', 'Couch']


In [66]:
response = client.start_face_search(
    Video={
        'S3Object': {
            'Bucket': '2020sd',
            'Name': 'cuhack.mp4'
        }
    },
    CollectionId='Collection',
    NotificationChannel={
        'SNSTopicArn': 'arn:aws:sns:us-east-1:749661538333:AmazonRekognitionExample1580010752252',
        'RoleArn': 'arn:aws:iam::749661538333:role/CURole'
    },
)

In [67]:
print(response)

{'JobId': '5347dc1e478b0eb9bff6e0ae2e868acd125d27d97174741a865bddb94820346d', 'ResponseMetadata': {'RequestId': 'a1d7f53b-ab7d-42d9-81d2-d6523b5a7f2d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Sun, 26 Jan 2020 06:30:01 GMT', 'x-amzn-requestid': 'a1d7f53b-ab7d-42d9-81d2-d6523b5a7f2d', 'content-length': '76', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [68]:
response = client.get_face_search(
    JobId='5347dc1e478b0eb9bff6e0ae2e868acd125d27d97174741a865bddb94820346d'
)

In [69]:
print(response) #nyes or no sumedha in video or not

{'JobStatus': 'SUCCEEDED', 'VideoMetadata': {'Codec': 'h264', 'DurationMillis': 6032, 'Format': 'QuickTime / MOV', 'FrameRate': 30.011274337768555, 'FrameHeight': 352, 'FrameWidth': 640}, 'Persons': [{'Timestamp': 0, 'Person': {'Index': 0, 'Face': {'BoundingBox': {'Width': 0.053440142422914505, 'Height': 0.12527327239513397, 'Left': 0.3424128592014313, 'Top': 0.27912279963493347}, 'Landmarks': [{'Type': 'eyeLeft', 'X': 0.37786638736724854, 'Y': 0.3377436697483063}, {'Type': 'eyeRight', 'X': 0.38936811685562134, 'Y': 0.3352155387401581}, {'Type': 'mouthLeft', 'X': 0.37678074836730957, 'Y': 0.38308611512184143}, {'Type': 'mouthRight', 'X': 0.3856108486652374, 'Y': 0.38006335496902466}, {'Type': 'nose', 'X': 0.39404332637786865, 'Y': 0.36168262362480164}], 'Pose': {'Roll': -13.249750137329102, 'Yaw': 58.147518157958984, 'Pitch': -28.11573600769043}, 'Quality': {'Brightness': 30.597442626953125, 'Sharpness': 60.49041748046875}, 'Confidence': 99.99976348876953}}, 'FaceMatches': []}, {'Times

In [77]:
type(response)

dict

In [92]:
print(len(response['Persons']))

6


In [97]:
for i in range(len(response['Persons'])):
        faces = (response['Persons'][i]["FaceMatches"])
if faces is True:
        print("face recognized") 
else:
        print("Person is not available.")


Person is not available.


In [98]:
import csv
import boto3
import pandas as pd
import cv2
import matplotlib.pyplot as plt 
import json
import sys
import time

with open("credentials.csv","r") as input:
    next(input)
    reader=csv.reader(input)
    for line in reader:
        access_key_id=line[2]
        secret_access_key=line[3]
        
photo="hotairbaloon.jpg"

client = boto3.client('rekognition', aws_access_key_id = access_key_id, aws_secret_access_key = secret_access_key, region_name='us-east-1')

with open(photo,'rb') as source_image:
    source_bytes=source_image.read()
    
response=client.detect_labels(Image={'Bytes':source_bytes},MaxLabels=2,MinConfidence=95)
print(f"detect label response: {response})


pic='sum.jpeg'
      
      
response=client.detect_labels(Image={'S3Object':{'Bucket':'2020sd','Name':pic}},MaxLabels=10)
print(response)

{'Labels': [{'Name': 'Ball', 'Confidence': 99.47676086425781, 'Instances': [], 'Parents': []}, {'Name': 'Balloon', 'Confidence': 99.47676086425781, 'Instances': [{'BoundingBox': {'Width': 0.4699590802192688, 'Height': 0.7395995855331421, 'Left': 0.07522430270910263, 'Top': 0.09918608516454697}, 'Confidence': 99.47676086425781}], 'Parents': [{'Name': 'Ball'}]}], 'LabelModelVersion': '2.0', 'ResponseMetadata': {'RequestId': '4db9fbc9-2a1e-4f23-9870-e7cff47c480a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Sun, 26 Jan 2020 08:38:16 GMT', 'x-amzn-requestid': '4db9fbc9-2a1e-4f23-9870-e7cff47c480a', 'content-length': '362', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
